In [1]:
import logging
import sys
import os
sys.path.append(r"C:\Users\pasqu\Desktop\progettoasnm")
sys.path.append(r"C:\Users\pasqua\Desktop\progettoasnm")

from Code.notebook.graph.GraphConstructor import GraphConstructor
from preprocessing.text_preprocessor import TextPreprocessor
from models.clustering import Clustering
from visualization.cluster_visualizer import ClusterVisualizer
from models.sentiment_analysis import SentimentAnalyzer
from visualization.sentiment_visualizer import SentimentVisualizer
from visualization.wordcloud_visualizer import WordCloudVisualizer
from visualization.lda_visualizer import LDAViz
from models.topic_modeling import TopicModeling

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pasqua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pasqua\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
c:\Users\pasqua\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.info("Inizio del processo principale.")
output_dir = r"../../../Code/notebook//Analisi della polarizzazione dei contenuti//output"
# Costruzione del grafo
graph_builder = GraphConstructor(
        followers_paths= ["../../data_extraction/followers.csv"],
        data_paths = ["../../data_extraction/total_post1.csv", "../../data_extraction/total_post2.csv", "../../data_extraction/total_post3.csv"],
        info_filepath="graph_info.json",
        centralities_filepath="centralities_info.json",
    )
graph_builder.build_graph()
graph = graph_builder.graph

# # Estrazione e pre-elaborazione dei testi
preprocessor = TextPreprocessor()
user_opinions = preprocessor.extract_user_opinions(graph_builder)

In [3]:
sentiment_scores = SentimentAnalyzer().extract_sentiments_from_graph(graph_builder)

c:\Users\pasqua\Desktop\progettoasnm\Code\notebook\Analisi della polarizzazione dei contenuti\models\sentiment_analysis.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sentiment_scores = df_data.groupby("thread_user_pk").apply(


In [19]:
# Clustering
clustering = Clustering(cluster_file="output/cluster_labels.pkl")
cluster_labels = clustering.cluster(user_opinions, method="dbscan")

In [21]:
# Visualizzazione dei cluster
cluster_visualizer = ClusterVisualizer(output_dir=output_dir)
cluster_visualizer.visualize(user_opinions, cluster_labels)

In [28]:
# Visualizzazione del sentiment
sentiment_visualizer = SentimentVisualizer(output_dir=output_dir)
# Visualizza la distribuzione del sentiment
sentiment_visualizer.visualize_sentiment_distribution(
sentiment_scores, cluster_labels
)

# Visualizza la mappa di calore sentiment vs temi
sentiment_visualizer.visualize_sentiment_vs_themes_heatmap(
sentiment_scores, user_opinions, cluster_labels
)

In [29]:
polarizing_words = clustering.identify_polarizing_themes(
user_opinions, cluster_labels
)
wordcloud_visualizer = WordCloudVisualizer()
wordcloud_visualizer.visualize(polarizing_words, output_dir, "Un")

# BiGram
polarizing_words = clustering.identify_polarizing_themes_bigram(
user_opinions, cluster_labels
)
wordcloud_visualizer.visualize(polarizing_words, output_dir, "Bi")


In [ ]:
# # Topic Modeling
topic_modeling = TopicModeling()
lda_model, dictionary, corpus = topic_modeling.perform_topic_modeling(
user_opinions, len(set(cluster_labels.values()))
)

# Visualizzazione dei temi
lda_visualizer = LDAViz()
lda_visualizer.visualize(lda_model, corpus, dictionary, output_dir=output_dir)

logging.info("Processo principale completato.")